In [26]:
import pandas as pd
import requests

In [6]:
DeCrescent = pd.read_csv("C:/Users/simon/OneDrive/Documents/CUA COURSES/ENGR 441 & 442/Prototype 2/DeCrescent/daily_DeCrescent_master.csv")
Feuerwehr = pd.read_csv("C:/Users/simon/OneDrive/Documents/CUA COURSES/ENGR 441 & 442/Prototype 2/Feuerwehr/daily_Feuerwehr_master.csv")
HLT1 = pd.read_csv("C:/Users/simon/OneDrive/Documents/CUA COURSES/ENGR 441 & 442/Prototype 2/HLT1/daily_HLT1_Klagenfurt_Lastenstrasse_master.csv")
MAWC = pd.read_csv("C:/Users/simon/OneDrive/Documents/CUA COURSES/ENGR 441 & 442/Prototype 2/MAWC CocaCola/daily_MAWC_master.csv")
Nugali = pd.read_csv("C:/Users/simon/OneDrive/Documents/CUA COURSES/ENGR 441 & 442/Prototype 2/Nugali/daily_Nugali_master.csv")
Pentaari = pd.read_csv("C:/Users/simon/OneDrive/Documents/CUA COURSES/ENGR 441 & 442/Prototype 2/Pentaari/daily_Pentaari_master.csv")
Schipper = pd.read_csv("C:/Users/simon/OneDrive/Documents/CUA COURSES/ENGR 441 & 442/Prototype 2/Schipper Power/daily_Schipper Power- Three_master.csv")
sirisD = pd.read_csv("C:/Users/simon/OneDrive/Documents/CUA COURSES/ENGR 441 & 442/Prototype 2/siris-D-500kW/daily_siris-D_master.csv")
ww = pd.read_csv("C:/Users/simon/OneDrive/Documents/CUA COURSES/ENGR 441 & 442/Prototype 2/Wannon Water/daily_Wannon Water_master.csv")
wray = pd.read_csv("C:/Users/simon/OneDrive/Documents/CUA COURSES/ENGR 441 & 442/Prototype 2/Wray Farms/daily_Wray_Farms_Solar_System_master.csv")

In [8]:
master_df = pd.concat([DeCrescent, Feuerwehr, HLT1, MAWC, Nugali, Pentaari, Schipper, sirisD, ww, wray], ignore_index=True)
master_df['Date'] = pd.to_datetime(master_df['Date'])
master_df = master_df.sort_values(by='Date')

In [10]:
print(master_df.columns)

Index(['Date', 'Generated', 'Efficiency', 'System Name', 'Location',
       'System Size (kW)'],
      dtype='object')


In [12]:
master_df['Generated'] = (
    master_df['Generated'].astype(str).str.replace('kWh', '', regex=False).str.replace(',', '')
    .astype(float)
)

In [14]:
master_df = master_df.rename(columns={'Generated': 'Generated (kWh)'})

In [18]:
master_df.to_csv("MASTER_system_data.csv", index=False)

In [20]:
systems = [
    {"name": "siris-D-500kW", "lat": 40.912535, "lon": 23.911660},
    {"name": "Schipper Power - Three", "lat": 52.418746, "lon": 5.401354},
    {"name": "Pentaari", "lat": 40.468917, "lon": 21.219645},
    {"name": "MAWC CocaCola", "lat": 4.375234, "lon": 73.648900},
    {"name": "Wannon Water", "lat": -38.358293, "lon": 142.442208},
    {"name": "DeCrescent", "lat": 2.915812, "lon": -73.705485},
    {"name": "Nugali", "lat": -26.677069, "lon": -49.182773},
    {"name": "HTL1 Klagenfurt", "lat": 46.620340, "lon": 14.315173},
    {"name": "Wray Farms", "lat": 52.800070, "lon": -0.159891},
    {"name": "Feuerwehr", "lat": 47.537083, "lon": 8.730144}
]

In [34]:
start_date = "2023-09-22"
end_date = "2025-03-30"

def fetch_weather(lat, lon, name):
    url = "https://archive-api.open-meteo.com/v1/archive"
    params = {
        "latitude": lat,
        "longitude": lon,
        "start_date": start_date,
        "end_date": end_date,
        "daily": "shortwave_radiation_sum,temperature_2m_mean,cloudcover_mean",
        "timezone": "auto"
    }

    response = requests.get(url, params=params)
    data = response.json()

    df = pd.DataFrame({
        "Date": data["daily"]["time"],
        "Solar_Irradiance": data["daily"]["shortwave_radiation_sum"],
        "Temperature_2m": data["daily"]["temperature_2m_mean"],
        "Cloud_Cover": data["daily"]["cloudcover_mean"],
    })

    df["System Name"] = name
    return df

In [36]:
weather_frames = []

for system in systems:
    print(f"Fetching weather for {system['name']}")
    df = fetch_weather(system["lat"], system["lon"], system["name"])
    weather_frames.append(df)

weather_df = pd.concat(weather_frames, ignore_index=True)
weather_df["Date"] = pd.to_datetime(weather_df["Date"])

Fetching weather for siris-D-500kW
Fetching weather for Schipper Power - Three
Fetching weather for Pentaari
Fetching weather for MAWC CocaCola
Fetching weather for Wannon Water
Fetching weather for DeCrescent
Fetching weather for Nugali
Fetching weather for HTL1 Klagenfurt
Fetching weather for Wray Farms
Fetching weather for Feuerwehr


In [38]:
merged_df = pd.merge(master_df, weather_df, how="left", on=["Date", "System Name"])

In [40]:
merged_df.head(100)

,Date,Generated (kWh),Efficiency,System Name,Location,System Size (kW),Solar_Irradiance,Temperature_2m,Cloud_Cover
0,2023-09-22,180.330,3.6066,Wray Farms,United Kingdom,50.0,13.88,12.6,49.0
1,2023-09-23,140.380,2.8076,Wray Farms,United Kingdom,50.0,12.45,12.9,41.0
2,2023-09-24,86.931,1.73862,Wray Farms,United Kingdom,50.0,7.79,16.9,97.0
3,2023-09-25,171.482,3.42964,Wray Farms,United Kingdom,50.0,13.06,17.4,59.0
4,2023-09-26,123.231,2.46462,Wray Farms,United Kingdom,50.0,9.27,16.1,74.0
...,...,...,...,...,...,...,...,...,...
95,2023-12-26,43.366,0.86732,Wray Farms,United Kingdom,50.0,3.16,6.4,52.0
96,2023-12-27,2.788,0.05576,Wray Farms,United Kingdom,50.0,0.39,8.6,91.0
97,2023-12-28,24.289,0.48578,Wray Farms,United Kingdom,50.0,2.47,9.9,75.0
98,2023-12-29,33.550,0.671,Wray Farms,United Kingdom,50.0,2.04,7.2,42.0


In [78]:
dc_weather = pd.read_csv("DeCrescent_Weather.csv")
htl1_weather = pd.read_csv("HTL1_Weather.csv")
mawc_weather = pd.read_csv("MAWC_Weather.csv")
sp3_weather = pd.read_csv("Schipper Power- Three_Weather.csv")

In [82]:
temp = pd.concat([dc_weather, htl1_weather, mawc_weather, sp3_weather], ignore_index=True)
temp = temp.rename(columns={"time": "Date", "shortwave_radiation_sum (MJ/m²)": "Solar_Irradiance", "cloud_cover_mean (%)": "Cloud_Cover", "temperature_2m_mean (°C)": "Temperature_2m"})
temp["Date"] = pd.to_datetime(temp["Date"])
print(temp.columns)

Index(['Date', 'Solar_Irradiance', 'Cloud_Cover', 'Temperature_2m',
       'System Name'],
      dtype='object')


In [118]:
# Step 1: Merge on Date and System Name — but avoid duplicate columns
merged = pd.merge(
    merged_df,
    temp,
    how="left",
    on=["Date", "System Name"],
    suffixes=("", "_new")
)

# Step 2: Fill missing values in the original columns with the _new columns
for col in ["Temperature_2m", "Cloud_Cover", "Solar_Irradiance"]:
    if f"{col}_new" in merged.columns:
        merged[col] = merged[col].combine_first(merged[f"{col}_new"])
        merged.drop(columns=[f"{col}_new"], inplace=True)

# Step 3: Done — your dataframe is now updated
merged_df = merged

In [ ]:
merged_df.to_csv("MASTER_system_and_weather.csv", index=False)